In [1]:
import sys, time
import numpy as np
import arkouda as ak

In [2]:
ak.connect(server="localhost", port=5555)

4.3.1
psp =  tcp://localhost:5555
connected to tcp://localhost:5555


In [3]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-06.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-07.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-08.hdf"]

In [4]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start','srcBytes','dstBytes']

In [5]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

CPU times: user 2.07 ms, sys: 1.74 ms, total: 3.81 ms
Wall time: 11.5 s


In [6]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

print(nfDF['srcBytes'])
print(nfDF['dstBytes'])

[118781 118783 118785 ... 156417 156417 156417]
[34111590301 34111676798 48764590996 ... 34111524045 34111524045 34112016235]
[34111773101 34112055194 34111367844 ... 34111501502 34111501502 34111802699]
[5507 3137 5060 ... 35576 53968 56827]
[46272 445 5060 ... 53 53 80]
[0.000000e+00 3.008100e+05 1.477041e+06 ... 7.500000e+01 6.900000e+01 3.990000e+02]
[1.042329e+09 2.504080e+05 0.000000e+00 ... 0.000000e+00 0.000000e+00 0.000000e+00]


In [7]:
st = nfDF['start']
print(ak.is_sorted(st))

True


In [ ]:
gt = ak.GroupBy(st)
u,c = gt.count()
print(u.size,u)
print(c.size,c)

36973 [118781 118783 118785 ... 156415 156416 156417]
36973 [1 1 19 ... 3028 2535 466]


In [ ]:
%time u,s = gt.aggregate(nfDF['srcIP'],'nunique')
print(u.size,u)
print(s.size,s)

In [ ]:
print(nfDF)

In [ ]:
print(ak.info(ak.AllSymbols))
print(nfDF['srcIP'].name)

In [ ]:
%time u,c = ak.unique(nfDF['srcIP'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = ak.unique(nfDF['dstIP'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = ak.unique(nfDF['srcPort'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
%time u,c = ak.unique(nfDF['dstPort'], return_counts=True)

In [ ]:
print(u.size,u)
print(c.size,c)

In [ ]:
print((nfDF['srcPort'] == 80).sum())
print((nfDF['dstPort'] == 80).sum())

In [ ]:
s = u[0:100] # pick off some dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

In [ ]:
s = u[0:100].to_ndarray() # pick off some dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

In [ ]:
a = nfDF['start']
aMin = a.min()
aMax = a.max()
print(aMin,aMax, aMax - aMin)
%time h = ak.histogram(a,bins=100)
print(h)
print(h.sum(), a.size)

In [ ]:
a = nfDF['start']
print(a.size, a)
print(ak.is_sorted(a))
%time iv = ak.argsort(a)
print(iv.size, iv)
b = a[iv]
print(b.size, b)
print(ak.is_sorted(b))

In [ ]:
a = nfDF['dstPort']
print(a.size, a)
print(ak.is_sorted(a))
%time iv = ak.argsort(a)
print(iv.size, iv)
b = a[iv]
print(b.size, b)
print(ak.is_sorted(b))

In [ ]:
#ak.shutdown()
#ak.disconnect()